In [ ]:
# write a script that shows stock put options for the next 14 days that have a high probability of profit

# To find stock put options with a high probability of profit, we typically look for options with a high likelihood of expiring out of the money (OTM) or with a high probability of the underlying stock price staying above the strike price. This can be estimated using various option pricing models such as the Black-Scholes model or implied volatility.

# Below is a script that uses the yfinance library to fetch historical stock data and the mibian library to calculate the Black-Scholes implied volatility and other option parameters. This script calculates the probability of profit (POP) for put options expiring in the next 14 days and filters out options with a high POP:

# In this script:

# We fetch historical stock data for the specified ticker.
# For each put option available, we calculate the implied volatility using the Black-Scholes model.
# Using the implied volatility and the current stock price, we estimate the probability of profit for each put option.
# We filter out put options with a high probability of profit (POP) and print the details of these options.
# Please note that the Black-Scholes model is just one way to estimate option prices and implied volatility. There are other models and factors to consider in option trading. Also, remember to consult with financial experts or conduct thorough research before making any trading decisions.




In [2]:
pip install mibian

  Preparing metadata (setup.py) ... done
  Created wheel for mibian: filename=mibian-0.1.3-py3-none-any.whl size=4024 sha256=51dfeb31c7581c9cfc93bcae9bd41225f9dc696f1cc95ee3f5e3125ed757a83d
  Stored in directory: /Users/ganderson/Library/Caches/pip/wheels/bb/8e/88/9401e806f2eaf5d2f4e775e598c27d7cc09bfb06998d075093
Successfully built mibian
Note: you may need to restart the kernel to use updated packages.


In [19]:
import yfinance as yf
from datetime import datetime, timedelta
from mibian import BS

def get_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

def calculate_implied_volatility(data, strike_price, expiration_date):
    # Get the current stock price (last closing price)
    current_stock_price = data['Close'].iloc[-1]
    
    # Calculate time to expiration in years
    time_to_expiration = (expiration_date - datetime.now()).days / 365.0
    
    # Calculate the implied volatility using Black-Scholes model
    c = BS([current_stock_price, strike_price, 0, time_to_expiration], callPrice=0)
    return c.impliedVolatility

def calculate_probability_of_profit(implied_volatility, strike_price, current_stock_price):
    # Calculate the probability of profit using the normal cumulative distribution function (CDF)
    d1 = (1 / (implied_volatility * (14/365) ** 0.5)) * ((current_stock_price - strike_price) / current_stock_price)
    return 1 - d1

def find_high_pop_put_options(data, days_to_expiration):
    current_stock_price = data['Close'].iloc[-1]
    expiration_date = datetime.now() + timedelta(days=days_to_expiration)
    high_pop_puts = []
    
    for i, option in data.iterrows():

        print(option)
        options = yf.Ticker("AAPL").option_chain("2024-02-23").puts
        print(options)
        
        implied_volatility = calculate_implied_volatility(data, options['Strike'], expiration_date)
        pop = calculate_probability_of_profit(implied_volatility, options['Strike'], current_stock_price)
        
        if pop > 0.7:  # Adjust threshold for high POP as needed
            high_pop_puts.append({
                'Contract Name': options['Contract Name'],
                'Strike': options['Strike'],
                'Implied Volatility': implied_volatility,
                'Probability of Profit': pop
            })
    
    return high_pop_puts

# Example usage
if __name__ == "__main__":
    ticker = "AAPL"  # Replace with the desired stock symbol
    start_date = datetime.now() - timedelta(days=365)
    end_date = datetime.now()
    days_to_expiration = 14

    # Get historical stock data
    stock_data = get_stock_data(ticker, start_date, end_date)

    # Filter high probability of profit put options
    high_pop_puts = find_high_pop_put_options(stock_data, days_to_expiration)

    # Print the high POP put options
    for option in high_pop_puts:
        print(option)


[*********************100%%**********************]  1 of 1 completed


Open         1.523500e+02
High         1.530000e+02
Low          1.508500e+02
Close        1.525500e+02
Adj Close    1.517401e+02
Volume       5.914410e+07
Name: 2023-02-17 00:00:00, dtype: float64
         contractSymbol             lastTradeDate  strike  lastPrice    bid  \
0   AAPL240223P00095000 2024-02-16 15:28:24+00:00    95.0       0.01   0.00   
1   AAPL240223P00100000 2024-02-09 20:56:33+00:00   100.0       0.01   0.00   
2   AAPL240223P00105000 2024-02-16 18:27:09+00:00   105.0       0.01   0.00   
3   AAPL240223P00110000 2024-02-09 19:40:48+00:00   110.0       0.01   0.00   
4   AAPL240223P00115000 2024-02-14 16:16:10+00:00   115.0       0.01   0.00   
5   AAPL240223P00120000 2024-02-08 20:21:13+00:00   120.0       0.01   0.00   
6   AAPL240223P00125000 2024-02-14 20:58:06+00:00   125.0       0.01   0.00   
7   AAPL240223P00130000 2024-02-16 20:56:26+00:00   130.0       0.01   0.00   
8   AAPL240223P00135000 2024-02-16 20:55:51+00:00   135.0       0.01   0.00   
9   AAPL2402

KeyError: 'Strike'